# Decision tree classifier: diabetes prediction

Absolutely minimal MVP (minimum viable product) solution.

## 1. Data acquisition

### 1.1. Load

In [2]:
import pandas as pd

# Load the data from the URL
data_df=pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv")

data_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
data_df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
data_df.info

<bound method DataFrame.info of      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   5

In [9]:
data_df.shape

(768, 9)

### 1.2. Train-test split

In [10]:
from sklearn.model_selection import train_test_split

# Separate features from labels
labels=data_df['Outcome']
features=data_df.drop('Outcome', axis=1)

# Split the data into training and testing features and labels
training_features, testing_features, training_labels, testing_labels=train_test_split(
    features,
    labels,
    test_size=0.2,
    random_state=315
)

## 2. EDA

### 2.1. Features

In [8]:
# Inspect the training features' data types
training_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 765 to 611
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    int64  
 2   BloodPressure             614 non-null    int64  
 3   SkinThickness             614 non-null    int64  
 4   Insulin                   614 non-null    int64  
 5   BMI                       614 non-null    float64
 6   DiabetesPedigreeFunction  614 non-null    float64
 7   Age                       614 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 43.2 KB


In [15]:
training_features.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.851792,120.605863,69.602606,21.307818,83.459283,32.215961,0.473090,33.245928
std,3.403173,31.483407,18.224136,16.055309,120.021358,7.706636,0.339908,11.742608
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,100.000000,64.000000,0.000000,0.000000,27.500000,0.239250,24.000000
50%,3.000000,117.000000,72.000000,24.000000,39.000000,32.350000,0.370500,29.000000
75%,6.000000,138.000000,80.000000,33.000000,130.000000,36.575000,0.628500,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,72.000000


All of the features are already numeric - we don't need to do anything for the model to run. Let's check the labels too.

### 2.2. Labels

In [11]:
training_labels.info()

<class 'pandas.core.series.Series'>
Index: 614 entries, 765 to 611
Series name: Outcome
Non-Null Count  Dtype
--------------  -----
614 non-null    int64
dtypes: int64(1)
memory usage: 9.6 KB


In [29]:
mean_glucose = training_features['Glucose'].mean()
mean_blood_pressure = training_features['BloodPressure'].mean()
print(training_features[['Glucose', 'BloodPressure']].head())

     Glucose  BloodPressure
765      121             72
74        79             75
733      106             56
740      120             80
0        148             72


In [33]:
import pandas as pd

data_df[['Glucose', 'BloodPressure']] = data_df[['Glucose', 'BloodPressure']].replace(0, pd.NA)

data_df['Glucose'] = data_df['Glucose'].fillna(data_df['Glucose'].mean())
data_df['BloodPressure'] = data_df['BloodPressure'].fillna(data_df['BloodPressure'].mean())


print(data_df[['Glucose', 'BloodPressure']].head())



   Glucose  BloodPressure
0    148.0           72.0
1     85.0           66.0
2    183.0           64.0
3     89.0           66.0
4    137.0           40.0


In [31]:
print("Training Set:")
print(training_features[['Glucose', 'BloodPressure']].head())

print("\nTesting Set:")
print(testing_features[['Glucose', 'BloodPressure']].head())

Training Set:
     Glucose  BloodPressure
765      121             72
74        79             75
733      106             56
740      120             80
0        148             72

Testing Set:
     Glucose  BloodPressure
238      164             84
27        97             66
90        80             55
447       95             80
107      144             58


Also already numeric - we can move right to training the model and setting a baseline performance result.

## 3. Training

In [14]:
from sklearn.tree import DecisionTreeClassifier

# Instantiate and train decision tree classifier
model=DecisionTreeClassifier(random_state=315)
fit_result=model.fit(training_features, training_labels)

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

# Instantiate the Random Forest Classifier
rf_model = RandomForestClassifier(random_state=315)

# Define the hyperparameter grid for RandomizedSearchCV
rf_param_dist = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

# Set up the RandomizedSearchCV with verbose=0 to suppress output
rf_random_search = RandomizedSearchCV(
    estimator=rf_model, 
    param_distributions=rf_param_dist,
    n_iter=100, 
    scoring='accuracy', 
    cv=5, 
    verbose=0,  # Suppress the output
    n_jobs=-1
)

# Fit the random search to the training data
rf_random_search.fit(training_features, training_labels)

# Get the best model from random search
best_rf_model = rf_random_search.best_estimator_

# Make predictions from the test set features using the best Random Forest model
predicted_labels_rf = best_rf_model.predict(testing_features)

# Score predictions based on accuracy
percent_accuracy_rf = accuracy_score(testing_labels, predicted_labels_rf) * 100
print(f'Random Forest Model is {percent_accuracy_rf:.1f}% accurate on the test data')

# Print the best hyperparameters found
print("Best hyperparameters for Random Forest:", rf_random_search.best_params_)



Random Forest Model is 76.0% accurate on the test data
Best hyperparameters for Random Forest: {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10}


/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
170 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
170 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/vscode/.lo

## 4. Evaluation

In [43]:
from sklearn.metrics import accuracy_score

# Make predictions from test set features
predicted_labels=model.predict(testing_features)

# Score predictions from accuracy
percent_accuracy=accuracy_score(testing_labels, predicted_labels) * 100
print(f'Model is {percent_accuracy:.1f}% accurate on the test data')

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

Ok, done! Absolutely minimal decision tree classifier using ~10 statements. From here, there is **a lot** we can and should do to improve. But now we can do so incrementally, while maintaining a running state. We can also tell if anything we do 'works' in the performance sense by comparing to our baseline result of ~75% accuracy.